In [1]:
import pandas as pd
import datetime

In [2]:
file_path_onedrive = '/Volumes/System/Users/weisun/OneDrive/AER work'
file_path_external = '/Volumes/EnergyData/AER'

In [3]:
# Get YearMonth to FY mapping dict
YearMonthFY = pd.read_excel(f'{file_path_external}/Reference_data/duidinfo_SRMC.xlsx',
                         sheet_name='YearMonthFY').set_index(['Year','Month'])['FY'].to_dict()

In [9]:
def read_and_process_aemo_gen_emission(file_path_onedrive,YearMonthFY,Year):
    temp_df = pd.read_csv(f'{file_path_onedrive}/Electricity emission/AEMO_CO2EII/CO2EII_SUMMARY_RESULTS_{Year}.CSV',
                          skiprows=1,
                          usecols=['SETTLEMENTDATE','REGIONID','TOTAL_SENT_OUT_ENERGY','TOTAL_EMISSIONS'],
                          parse_dates=['SETTLEMENTDATE'],dayfirst=True)\
                .dropna(subset=['REGIONID'])\
                .rename(columns={'TOTAL_SENT_OUT_ENERGY':'ACTUAL_GEN',
                                 'TOTAL_EMISSIONS':'EMISSION'})
    temp_df = temp_df[temp_df['REGIONID'].isin(['NSW1','QLD1','VIC1','SA1','TAS1'])]
    
    temp_df['SETTLEMENTDATE'] = pd.to_datetime(temp_df['SETTLEMENTDATE'])
    temp_df['Year'] = temp_df['SETTLEMENTDATE'].apply(lambda x: x.year)
    temp_df['Month'] = temp_df['SETTLEMENTDATE'].apply(lambda x:x.month)
    temp_df['FY'] = temp_df.apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
    temp_df = temp_df.groupby(by=['REGIONID','FY','Year','Month'],as_index=False)[['ACTUAL_GEN','EMISSION']].sum()
    return temp_df

In [13]:
aemo_emission_df = pd.DataFrame()
for year in range(2017,2024):
    temp_df = read_and_process_aemo_gen_emission(file_path_onedrive,YearMonthFY,year)
    aemo_emission_df = pd.concat([aemo_emission_df,temp_df])
aemo_emission_df = aemo_emission_df.groupby(by=['REGIONID','FY','Year','Month'],as_index=False)[['ACTUAL_GEN','EMISSION']].sum()

/var/folders/xt/rmsbqlvs2vz9v3p1pyby_p0w0000gn/T/ipykernel_4905/3272811214.py:2: UserWarning: Parsing dates in %Y/%m/%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  temp_df = pd.read_csv(f'{file_path_onedrive}/Electricity emission/AEMO_CO2EII/CO2EII_SUMMARY_RESULTS_{Year}.CSV',
/var/folders/xt/rmsbqlvs2vz9v3p1pyby_p0w0000gn/T/ipykernel_4905/3272811214.py:2: UserWarning: Parsing dates in %Y/%m/%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  temp_df = pd.read_csv(f'{file_path_onedrive}/Electricity emission/AEMO_CO2EII/CO2EII_SUMMARY_RESULTS_{Year}.CSV',
/var/folders/xt/rmsbqlvs2vz9v3p1pyby_p0w0000gn/T/ipykernel_4905/3272811214.py:2: UserWarning: Parsing dates in %Y/%m/%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  temp_df = pd.read_csv(f'{file_path_onedrive}/Electricit

In [17]:
aemo_emission_df.to_csv(f'{file_path_onedrive}/Electricity emission/aemo_generation_emission_ws.csv',index=False)